In [1]:
import sys
# 프로젝트 루트 직접 지정
sys.path.insert(0, r"c:\Users\tjrrj\vscode\doritest")

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
import pandas as pd
from typing import Dict, List, Union

AggFunc = Union[str, List[str]]


def agg_df(
    df: pd.DataFrame,
    group_cols: List[str],
    agg_map: Dict[str, AggFunc],
    *,
    reset_index: bool = True,
) -> pd.DataFrame:
    """
    DataFrame 그룹 집계 함수

    Args:
        df: 원본 DataFrame
        group_cols: 그룹 기준 컬럼 리스트
        agg_map: 집계 규칙
            예)
            {
                "매출": "sum",
                "주문수": "sum",
                "객단가": "mean",
                "재고": ["min", "max"]
            }
        reset_index: index 초기화 여부

    Returns:
        집계된 DataFrame
    """

    # ===== 컬럼 존재 여부 체크 =====
    missing_cols = set(group_cols) - set(df.columns)
    missing_aggs = set(agg_map.keys()) - set(df.columns)

    if missing_cols:
        raise ValueError(f"그룹 컬럼이 존재하지 않습니다: {missing_cols}")
    if missing_aggs:
        raise ValueError(f"집계 컬럼이 존재하지 않습니다: {missing_aggs}")

    # ===== groupby aggregation =====
    result = df.groupby(group_cols).agg(agg_map)

    # ===== 컬럼 평탄화 (멀티 인덱스 제거) =====
    if isinstance(result.columns, pd.MultiIndex):
        result.columns = [
            "_".join(col).rstrip("_")
            for col in result.columns.values
        ]

    if reset_index:
        result = result.reset_index()

    return result


In [16]:
import pandas as pd

df = pd.read_excel("D:\Download\coupang_eats_2025-11.xlsx", header=1).head(2)

<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
C:\Users\tjrrj\AppData\Local\Temp\ipykernel_24868\3234733784.py:3: SyntaxWarning: invalid escape sequence '\D'
  df = pd.read_excel("D:\Download\coupang_eats_2025-11.xlsx", header=1).head(2)
c:\Users\tjrrj\vscode\doritest\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [17]:
df

,일자,시간,주문번호,유형,상세내역,브랜드명,상점명,결제방식,거래유형,총금액,...,Unnamed: 33,공급가액,부가세액,총액,산정전.3,Unnamed: 38,산정후.3,Unnamed: 40,Unnamed: 41,Unnamed: 42
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,총액,NaN,NaN,NaN,기본요금,프로모션,NaN,NaN,NaN,NaN
1,2025-11-01,2025-11-01 13:41:28,2DEKBH,배달,김치찜 먹고 싶을 땐 묵은지 도리탕,NaN,닭도리탕 전문 도리당 가락점,신용카드,결제,27100.0,...,6570,2410.0,241.0,2651.0,13879,13879,13879.0,NaN,0.0,0.0


In [18]:
aggregate_df(df,
             group_cols=["유형", "거래유형"],
    agg_map={
        "총금액": "sum",
        "공급가액": "sum",
        "부가세액": ["mean", "max", "min"],
    }
)

,유형,거래유형,총금액_sum,공급가액_sum,부가세액_mean,부가세액_max,부가세액_min
0,배달,결제,27100.0,2410.0,241.0,241.0,241.0


# 코드